In [1]:
# banks_project.py

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime

# Task 1: Logging Progress
def log_progress(message):
    timestamp_format = '%Y-%b-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("code_log.txt", "a") as f:
        f.write(timestamp + ' : ' + message + '\n')


# Task 2: Extract Data from URL
def extract(url, table_attribs):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

    # Locate the correct table under 'By market capitalization'
    tables = soup.find_all("table", {"class": "wikitable"})
    target_table = tables[0]  # First wikitable is the correct one

    rows = target_table.find_all("tr")
    for row in rows[1:]:  # skip header
        cols = row.find_all("td")
        if len(cols) >= 2:
            name = cols[1].text.strip()
            mc_usd = cols[2].text.strip().replace("\n", "").replace(",", "")
            try:
                mc_usd = float(mc_usd)
                df = pd.concat([df, pd.DataFrame([[name, mc_usd]], columns=table_attribs)], ignore_index=True)
            except ValueError:
                continue
    return df


# Task 3: Transform Data using exchange rates
def transform(df, exchange_rate_path):
    rates = pd.read_csv(exchange_rate_path, index_col=0)
    usd_to_gbp = rates.loc["GBP"].values[0]
    usd_to_eur = rates.loc["EUR"].values[0]
    usd_to_inr = rates.loc["INR"].values[0]

    df["MC_GBP_Billion"] = np.round(df["MC_USD_Billion"] * usd_to_gbp, 2)
    df["MC_EUR_Billion"] = np.round(df["MC_USD_Billion"] * usd_to_eur, 2)
    df["MC_INR_Billion"] = np.round(df["MC_USD_Billion"] * usd_to_inr, 2)
    print (df['MC_EUR_Billion'][4])
    return df


# Task 4: Load to CSV
def load_to_csv(df, path):
    df.to_csv(path, index=False)


# Task 5: Load to DB
def load_to_db(df, db_name, table_name):
    connection = sqlite3.connect(db_name)
    df.to_sql(table_name, connection, if_exists='replace', index=False)
    connection.close()


# Task 6: Run Queries
def run_queries(db_name, query):
    connection = sqlite3.connect(db_name)
    print(f"\nExecuting Query: {query}")
    result = pd.read_sql(query, connection)
    print(result)
    connection.close()


# --------------- MAIN EXECUTION -------------------

# Configuration
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
exchange_rate_csv = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
table_attribs = ["Name", "MC_USD_Billion"]
output_csv_path = './Largest_banks_data.csv'
db_name = 'Banks.db'
table_name = 'Largest_banks'

# Execution steps
log_progress("ETL Job Started")

df = extract(url, table_attribs)
log_progress("Data Extraction Complete")

df = transform(df, exchange_rate_csv)
log_progress("Data Transformation Complete")

load_to_csv(df, output_csv_path)
log_progress("Data Loaded to CSV")

load_to_db(df, db_name, table_name)
log_progress("Data Loaded to Database")

run_queries(db_name, f"SELECT * FROM {table_name} ORDER BY MC_USD_Billion DESC LIMIT 5")
log_progress("Query Execution Complete")

# Execute Query 1: Print the contents of the entire table
query_1 = "SELECT * FROM Largest_banks"
run_queries(db_name, query_1)
log_progress("Query 1 executed: Full table printed")

# Execute Query 2: Print the average market capitalization in GBP
query_2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_queries(db_name, query_2)
log_progress("Query 2 executed: Average Market Cap in GBP printed")

# Execute Query 3: Print names of top 5 banks
query_3 = "SELECT Name from Largest_banks LIMIT 5"
run_queries(db_name, query_3)
log_progress("Query 3 executed: Top 5 bank names printed")


log_progress("ETL Job Finished")


/tmp/ipython-input-1-1960250429.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([[name, mc_usd]], columns=table_attribs)], ignore_index=True)


146.86

Executing Query: SELECT * FROM Largest_banks ORDER BY MC_USD_Billion DESC LIMIT 5
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75  
3          149.43        13328.41  
4          146.86        13098.63  

Executing Query: SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                      